# **Hello**

****This is a beginner friendly intermediate level notebook ;)****

****Please do comment and spread your knowledge. Enlighten me and others.****

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e7/sample_submission.csv
/kaggle/input/playground-series-s5e7/train.csv
/kaggle/input/playground-series-s5e7/test.csv


****So here we start.****

In [2]:
#importing useful libraries
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
#Loading Datasets

X_full = pd.read_csv('../input/playground-series-s5e7/train.csv')
X_test = pd.read_csv('../input/playground-series-s5e7/test.csv')

X_full.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,0,0.0,No,6.0,4.0,No,15.0,5.0,Extrovert
1,1,1.0,No,7.0,3.0,No,10.0,8.0,Extrovert
2,2,6.0,Yes,1.0,0.0,NaN,3.0,0.0,Introvert
3,3,3.0,No,7.0,3.0,No,11.0,5.0,Extrovert
4,4,1.0,No,4.0,4.0,No,13.0,NaN,Extrovert


Some columns have categorical data and for now I don't have much expertize in dealing with categorical data, hence it is better to turn them into numerical ones.

In [4]:
#updating categorical data into binary terms

def turn_to_numeric(Dataframe, column, string):
    new = Dataframe[column] == string
    new += 0
    Dataframe[column] = new
# 1 means Yes, 0 implies No
turn_to_numeric(X_full, 'Stage_fear', 'Yes')
turn_to_numeric(X_full, 'Drained_after_socializing', 'Yes')
turn_to_numeric(X_test, 'Stage_fear', 'Yes')
turn_to_numeric(X_test, 'Drained_after_socializing', 'Yes')

# 1 means Exrovert and 0 refers to Introvert
turn_to_numeric(X_full, 'Personality', 'Extrovert')

Finally splitting the transformed data.

In [5]:
X_full.head()
y = X_full["Personality"]
X = X_full.drop(["id","Personality"], axis = 1)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

Imputation for the NAN values.
Since the test data also contains NAN values, it is not a good idea to drop that data.

I will also check in some other version for putting 0 or something like that for NAN values. In that case, I will need to increase the weights of other values.

In [7]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [8]:
##### model creation
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 150,random_state = 0)
model.fit(imputed_X_train, y_train)

RandomForestRegressor(n_estimators=150, random_state=0)

Model is created, let's now test it.

In [42]:
test_pred = model.predict(imputed_X_valid)
for i in range(len(test_pred)):
    test_pred[i] = 1 if test_pred[i]>=0.57 else 0
test_pred

array([0., 1., 1., ..., 0., 0., 1.])

Our model made really nice predictions, but they were all values ranging from 0 to 1. While we just wanted only two values that are 0 and 1. So we converted those having value 0.5 and above to 1.

(The model performed better if instead of 0.5, 0.57 was choosen.)

Nice!! But we will require some thing to measure if our model really worked well

In [43]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_valid, test_pred)

0.034817813765182185

Great!!! Now let's make our model work on real test data.

In [11]:
X_test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency
0,18524,3.0,0,7.0,4.0,0,6.0,NaN
1,18525,NaN,1,0.0,0.0,1,5.0,1.0
2,18526,3.0,0,5.0,6.0,0,15.0,9.0
3,18527,3.0,0,4.0,4.0,0,5.0,6.0
4,18528,9.0,1,1.0,2.0,1,1.0,1.0
...,...,...,...,...,...,...,...,...
6170,24694,3.0,0,5.0,5.0,0,9.0,6.0
6171,24695,8.0,1,2.0,1.0,1,0.0,0.0
6172,24696,2.0,0,4.0,3.0,0,9.0,7.0
6173,24697,3.0,0,4.0,4.0,0,11.0,9.0


The test data has some missing values, so let's impute them.

In [14]:
imputed_X_test = pd.DataFrame(my_imputer.fit_transform(X_test))
imputed_X_test.columns = X_test.columns
imputed_X_test = imputed_X_test.drop(['id'], axis = 1)
final_prediction = model.predict(imputed_X_test)

final_prediction

array([1.        , 0.00666667, 0.96666667, ..., 1.        , 1.        ,
       0.        ])

Final Predictions are made, now let's convert it into the asked format.

In [44]:
list = []
for i in range(len(final_prediction)):
    if final_prediction[i]>=0.57:
        list.append("Extrovert")
    else :
        list.append("Introvert")

output = pd.DataFrame({'id': X_test.id,
                       'Personality': list})
output.to_csv('submission.csv', index=False)

Awesome!! That's it, just one more line of code. (To see how our output looks like.

In [45]:
output

,id,Personality
0,18524,Extrovert
1,18525,Introvert
2,18526,Extrovert
3,18527,Extrovert
4,18528,Introvert
...,...,...
6170,24694,Extrovert
6171,24695,Introvert
6172,24696,Extrovert
6173,24697,Extrovert
